In [3]:
#importing and cleaning data, leaving what i need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
anime = pd.read_csv('data/anime-dataset-2023.csv')


In [4]:
def clean_data(anime):
    #dropping columns
    anime = anime.drop(columns=['Members', 'Scored By', 'Favorites', 'Source', 'Licensors', 'Producers', 'Premiered', 'English name','Studios', 'Duration', 'Rating', 'Status', 'Other name', 'Aired'])
    #renaming a column
    anime = anime.rename(columns={'Synopsis': 'Description'})
    return anime

anime_clean = clean_data(anime.copy())
anime_clean.head()

,anime_id,Name,Score,Genres,Description,Type,Episodes,Rank,Popularity,Image URL
0,1,Cowboy Bebop,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,41.0,43,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,189.0,602,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,328.0,246,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,2764.0,1795,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,4240.0,5126,https://cdn.myanimelist.net/images/anime/7/215...


In [5]:
#concatenating the columns together to make it easier for model to compare data
def clean_data(anime_clean):
    #first replacing and filling in data that has "unknown"
    anime_clean['Episodes'] = anime_clean['Episodes'].str.replace("UNKNOWN", "1000+", case=False, regex=False)
    #changing column type to string
    anime_clean = anime_clean.astype({'Episodes': 'string', 'Popularity': 'string', 'Score': 'string', 'Rank': 'string'})
    anime_clean['tags'] = anime_clean['Description'] + anime_clean['Genres'] + anime_clean['Type'] + anime_clean['Episodes'] + anime_clean['Popularity'] + anime_clean['Score'] + anime_clean['Rank'] 
    anime_clean = anime_clean[['anime_id','Name','tags']]

    anime_clean['tags'] = anime_clean['tags'].apply(lambda x:x.lower())
    return anime_clean

anime_clean = clean_data(anime_clean.copy())
anime_clean.head(3)


,anime_id,Name,tags
0,1,Cowboy Bebop,"crime is timeless. by the year 2071, humanity ..."
1,5,Cowboy Bebop: Tengoku no Tobira,"another day, another bounty—such is the life o..."
2,6,Trigun,"vash the stampede is the man with a $$60,000,0..."


In [61]:
#making data into vector to use cosine similarity to help model
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000, stop_words= 'english')

vector = cv.fit_transform(anime_clean['tags']).toarray()

vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [95]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity
similarity.shape

(24905, 24905)

In [39]:
anime_clean[anime_clean['Name'] == 'Trigun'].index[0]

np.int64(2)

In [87]:

#making it recommend and give position of data similarity to list 5 recommendations in a sorted list
def recommend(anime):
    index = anime_clean[anime_clean['Name'] == anime].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True , key= lambda x: x[1])
    for i in distances[1:30]:
        print(anime_clean.iloc[i[0]].Name)

In [88]:
recommend('Naruto')

Naruto (2023)
Boruto: Naruto Next Generations
Naruto: Shippuuden
Naruto: Shippuuden Movie 4 - The Lost Tower
Boruto: Naruto the Movie
Naruto: Honoo no Chuunin Shiken! Naruto vs. Konohamaru!!
Naruto: Shippuuden - Shippuu! "Konoha Gakuen" Den
Naruto: Shippuuden Movie 5 - Blood Prison
Naruto: Shippuuden Movie 2 - Kizuna
Naruto (Shinsaku Anime)
Juliet
Mayoiga
Naruto: Shippuuden Movie 6 - Road to Ninja
Naruto: Takigakure no Shitou - Ore ga Eiyuu Dattebayo!
Road of Naruto
Kimetsu no Yaiba: Katanakaji no Sato-hen
The Last: Naruto the Movie
Boruto: Naruto the Movie - Naruto ga Hokage ni Natta Hi
Toriko 3D: Kaimaku Gourmet Adventure!!
Shi Cao Lao Long Bei Guan Yi E Long Zhi Ming
Naruto: Shippuuden Movie 1
Yamato Takeru: After War
Kogane no Hana
Kamisama no Inai Nichiyoubi
Kishin Douji Zenki Gaiden: Anki Kitan
Mononoke Hime
Naruto Soyokazeden Movie: Naruto to Mashin to Mitsu no Onegai Dattebayo!!
Higurashi no Naku Koro ni
Heisei Inu Monogatari Bow Movie


In [89]:
recommend('Bleach')


Bleach: Sennen Kessen-hen
Bleach Movie 3: Fade to Black - Kimi no Na wo Yobu
Bleach Movie 4: Jigoku-hen
Bleach Movie 1: Memories of Nobody
Tokyo Mew Mew New ♡
Bleach: The Sealed Sword Frenzy
Yume-iro Pâtissière SP Professional
Tokyo Mew Mew
Ichigo Mashimaro Encore
Tokyo Mew Mew New ♡ 2nd Season
Aikatsu! Movie
Umi no Triton
Wu Ying Sanqian Dao
Umineko no Naku Koro ni
Yuan Long
Bleach Movie 2: The DiamondDust Rebellion - Mou Hitotsu no Hyourinmaru
Yume-iro Pâtissière
Glass no Usagi
Ling Yu
Yozakura-san Chi no Daisakusen
Gokiburi-chan
Barbapapa Sekai wo Mawaru
Doubutsu Mura no Daisodou
Jue Ji
Crayon Shin-chan Movie 23: Ora no Hikkoshi Monogatari - Saboten Daisuugeki
Yadamon
Da Yu Hai Tang (Movie)
Bai Lian Cheng Shen
Maryuu Senki


In [96]:
import pickle

pickle.dump(anime_clean, open('models/anime_list.pkl', 'wb'))
pickle.dump(similarity, open('models/similarity.pkl', 'wb'))

In [1]:
# Function to split the file into parts
def split_file(file_path, chunk_size=100 * 1024 * 1024):  # Split into 500MB chunks
    with open(file_path, 'rb') as f:
        i = 0
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            with open(f'similarity_part_{i}.pkl', 'wb') as chunk_file:
                chunk_file.write(chunk)
            i += 1

# File path of original large similarity.pkl
file_path = 'models/similarity.pkl'
split_file(file_path)

print("File split successfully!")

File split successfully!
